## OpenSn Tutoiral: 3D 9-Pebble BCC Lattice Cell

This script performs a steady-state DFEM k-eigenvalue calculation for a nine-pebble geometry representing an infinite lattice. The corner pebbles are assigned homogenized multigroup cross sections that differ from those of the center pebble. The gap material consists of low-density helium. The unstructured tetrahedral mesh can be created using the jupyter notebook,

opensn/doc/source/tutorials/lbs/9pebs/mesh/gmsh_9peb.ipynb

![Visualization of 9-pebble mesh](images/9peb_mesh_visualization.png)

This figure shows the nine-pebble mesh, with the helium gap material hidden for visual clarity. Each colored region corresponds to a distinct mesh subdomain or block, which can be assigned a specific cross-section representing the material properties for that region.

The cell below begins by setting up MPI and importing all useful Python and OpenSn modules. An executable form of this script,

opensn/doc/source/tutorials/lbs/9pebs/9_pebs_dfem_9grp.py

is designed to be run in parallel using mpirun.

In [ ]:
import os
import sys
import re

# ---------------------------------------------------------
# MPI + OpenSn Python bindings setup
# ---------------------------------------------------------
if "opensn_console" not in globals():
    from mpi4py import MPI
    size = MPI.COMM_WORLD.size
    rank = MPI.COMM_WORLD.rank

    # Add OpenSn Python module path (adjust if necessary)
    sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "../../../../../")))

    # Import OpenSn components
    from pyopensn.aquad import GLCProductQuadrature3DXYZ
    from pyopensn.solver import DiscreteOrdinatesSolver
    from pyopensn.fieldfunc import FieldFunctionGridBased
    from pyopensn.solver import NonLinearKEigen


## Load the 9-Pebble Mesh

This step is important not only to load the mesh file, but also to partition it for parallel execution. Each mpi process will be assigned a subset or partition of the mesh. The partitioner (parmetis) uses this information to decide how to split the mesh for parallel computation. The image below shows an example where the 9-pebble domain is partitioned between 16 mpi tasks.

![Visualization of 9-pebble mesh partition](images/9peb_mesh_partitioned.png)



In [ ]:
mesh_file = "mesh/msh_meshes/9_peb.msh"

# Load mesh and partition it for parallel execution
meshgen = FromFileMeshGenerator(
    filename=mesh_file,
    partitioner=PETScGraphPartitioner(type='parmetis')
)

grid = meshgen.Execute()

# Export mesh to Paraview format for inspection
grid.ExportToPVTU("9pebs")


## Load Multigroup Cross Sections

Here, there are three cross section files corresponding to three separate materials, the corner pebbles, the center pebble, and the helium gap.

In [ ]:
xss = []
for m in range(3):
    xss.append(MultiGroupXS())

# Store cross section data as a list
xss[0].LoadFromOpenSn("xs_files/xs_p_1.xs")  # corner pebbles
xss[1].LoadFromOpenSn("xs_files/xs_p_5.xs")  # center pebble
xss[2].LoadFromOpenSn("xs_files/xs_50.xs")   # helium gap

# Store the number of energy groups
num_groups = xss[0].num_groups
print("Number of energy groups:", num_groups)


## Map Cross Sections to Mesh Block IDs

Assign XS to each mesh region based on block IDs. Note that a single material may be replicated across several block IDs.

In [ ]:

xs_map = []
xs_map.append({"block_ids": [0, 1, 2, 3, 5, 6, 7, 8], "xs": xss[0]})  # corner pebbles
xs_map.append({"block_ids": [4], "xs": xss[1]})                       # center pebble
xs_map.append({"block_ids": [9], "xs": xss[2]})                       # coolant gap


## Angular Quadrature:

Declare Gauss-Legendre Chebyshev product quadrature

In [ ]:

# Choose S4 quadrature (can try S2 or S8)
pquad = GLCProductQuadrature3DXYZ(4, 8)

## Discrete Ordiantes (SN) Solver Setup:

Configure solver with mesh, groups, quadrature, and material map which were all set up in the previous sections.

In [ ]:

phys = DiscreteOrdinatesSolver(
    mesh=grid,
    num_groups=num_groups,
    groupsets=[
        {
            "groups_from_to": (0, num_groups - 1),
            "angular_quadrature": pquad,
            "inner_linear_method": "petsc_gmres",
            "l_max_its": 10,
            "l_abs_tol": 1.0e-7,
            "angle_aggregation_type": "single",
            "angle_aggregation_num_subsets": 1,
        },
    ],
    xs_map=xs_map,
    options={
        "boundary_conditions": [
            {"name": "xmin", "type": "reflecting"},
            {"name": "xmax", "type": "reflecting"},
            {"name": "ymin", "type": "reflecting"},
            {"name": "ymax", "type": "reflecting"},
            {"name": "zmin", "type": "reflecting"},
            {"name": "zmax", "type": "reflecting"}
        ],
        "scattering_order": 2,
        "verbose_outer_iterations": True,
        "verbose_inner_iterations": True,
        "power_field_function_on": True,
        "power_default_kappa": 1.0,
        "power_normalization": 2000.0,
        "save_angular_flux": True,
    },
    sweep_type="CBC",
)


## Nonlinear k-Eigenvalue Solver

Initialize and run the criticality calculation.

In [ ]:

k_solver = NonLinearKEigen(
    lbs_solver=phys,
    nl_max_its=50,
    nl_abs_tol=1.0e-10,
    nl_rel_tol=1.0e-10,
    l_max_its=20,
    num_initial_power_iterations=2,
)

k_solver.Initialize()
k_solver.Execute()


## Export Scalar Flux to VTK

This code exports the scalar flux field functions computed by the solver to VTK format for visualization and post-processing. By calling GetScalarFieldFunctionList, only the scalar flux fields are retrieved, omitting angular fluxes to reduce file size and simplify analysis. ExportMultipleToVTK then writes these fields to files with the specified basename, creating one or more VTK files suitable for ParaView or other visualization tools. This allows the user to inspect the spatial distribution of neutron flux throughout the nine-pebble geometry, verify solver results, and perform further analyses such as integrating flux to compute reaction rates or visualizing cross-section assignments across different mesh blocks.

In [ ]:

fflist = phys.GetScalarFieldFunctionList(only_scalar_flux=True)
vtk_basename = "9pebs_dfem_9grp_out"

FieldFunctionGridBased.ExportMultipleToVTK(fflist, vtk_basename)


Exporting the scalar flux field to a VTK-compatible mesh enables detailed postprocessing and quantitative analysis in visualization tools such as ParaView. The image below shows the spatial distribution of thermal flux (group 7 of 9) in the nine-pebble problem.

![Visualization g7 Flux Solution](images/9peb_flux_solution_g7.png)